In [1]:
import os
import pathlib
from dotenv import load_dotenv

# Load some standard environment variables from a dot-env file, if it exists.
# If no such file can be found, does not fail, and so allows these environment vars to
# be populated in some other way
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

Set session variable CATALOG to make query terms much more compact

In [2]:
import trino
from sqlalchemy.engine import create_engine

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ['TRINO_USER'],
    host = os.environ['TRINO_HOST'],
    port = os.environ['TRINO_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

import pandas as pd

The EPA's GHG Reporting Program (GHGRP) seems to be a gold standard in terms of creating a bottoms-up list that's good enough to play a major role in tops-down estimates.

In [3]:
qres = engine.execute(f"show tables in epa_ghgrp")
display(qres.fetchall())

[('co2_injection',),
 ('direct_emitters',),
 ('gathering_boosting',),
 ('geologic_sequestration_of_co2',),
 ('ldc_direct_emissions',),
 ('onshore_oil_gas_prod',),
 ('parent_company',),
 ('sf6_from_elec_equip',),
 ('suppliers',),
 ('transmission_pipelines',)]

`Direct_Emitters` are the lion's share of CO2 _emissions_.  `Suppliers` tracks fuels and products which, when used as intended, will create GHG emissions (by direct emitters or others).

In [4]:
qres = engine.execute(f"describe epa_ghgrp.direct_emitters")
display(qres.fetchall())

[('facility_id', 'bigint', '', ''),
 ('frs_id', 'varchar', '', ''),
 ('facility_name', 'varchar', '', ''),
 ('city', 'varchar', '', ''),
 ('state', 'varchar', '', ''),
 ('zip_code', 'varchar', '', ''),
 ('address', 'varchar', '', ''),
 ('county', 'varchar', '', ''),
 ('latitude', 'double', '', ''),
 ('longitude', 'double', '', ''),
 ('primary_naics_code', 'varchar', '', ''),
 ('latest_reported_industry_type_subparts', 'varchar', '', ''),
 ('latest_reported_industry_type_sectors', 'varchar', '', ''),
 ('total_reported_direct_emissions', 'double', '', ''),
 ('year', 'timestamp(3)', '', '')]

In [5]:
qres = engine.execute(f"select year, sum(total_reported_direct_emissions)/1000000000, '<- GtCO2e' as metric from epa_ghgrp.direct_emitters group by year order by year desc")
display(qres.fetchall())

[('2020-01-01 00:00:00.000', 2.4003350171144833, '<- GtCO2e'),
 ('2019-01-01 00:00:00.000', 2.6265320985932563, '<- GtCO2e'),
 ('2018-01-01 00:00:00.000', 2.7794706371794247, '<- GtCO2e'),
 ('2017-01-01 00:00:00.000', 2.73584075489731, '<- GtCO2e'),
 ('2016-01-01 00:00:00.000', 2.8051047475443416, '<- GtCO2e'),
 ('2015-01-01 00:00:00.000', 2.9394435833242407, '<- GtCO2e'),
 ('2014-01-01 00:00:00.000', 3.0840687932163457, '<- GtCO2e'),
 ('2013-01-01 00:00:00.000', 3.0732138962733337, '<- GtCO2e'),
 ('2012-01-01 00:00:00.000', 3.0580757925988027, '<- GtCO2e'),
 ('2011-01-01 00:00:00.000', 3.2075829940028187, '<- GtCO2e')]

Here's a look at how they stack up (from a Database perspective--we should also look at this in Super Set).

In [6]:
qres = engine.execute(f"""select count (*), latest_reported_industry_type_sectors, sum(total_reported_direct_emissions)/1000000, 'MtCO2e' as metric
from epa_ghgrp.direct_emitters
where year>=DATE('2019-01-01') and year<DATE('2020-01-01')
group by latest_reported_industry_type_sectors
order by sum(total_reported_direct_emissions) desc
""")
display(qres.fetchall())

[(1338, 'Power Plants', 1593.3026511495946, 'MtCO2e'),
 (371, 'Minerals', 112.04449316399996, 'MtCO2e'),
 (328, 'Chemicals', 106.40426483600002, 'MtCO2e'),
 (1066, 'Other', 101.35302250774214, 'MtCO2e'),
 (1187, 'Petroleum and Natural Gas Systems', 101.03201285843231, 'MtCO2e'),
 (1204, 'Waste', 100.62444679977303, 'MtCO2e'),
 (281, 'Metals', 69.49411383299993, 'MtCO2e'),
 (69, 'Petroleum Product Suppliers,Refineries', 65.079994478, 'MtCO2e'),
 (41, 'Chemicals,Petroleum Product Suppliers,Refineries', 64.019038494, 'MtCO2e'),
 (29, 'Chemicals,Suppliers of CO2', 27.43582412000001, 'MtCO2e'),
 (84, 'Pulp and Paper,Waste', 24.953431806000008, 'MtCO2e'),
 (7, 'Power Plants,Waste', 24.875284230000002, 'MtCO2e'),
 (11, 'Chemicals,Waste', 17.522490718, 'MtCO2e'),
 (10, 'Other,Power Plants', 17.315702882, 'MtCO2e'),
 (112, 'Other,Waste', 17.100924523999986, 'MtCO2e'),
 (3, 'Power Plants,Suppliers of CO2', 14.904814286, 'MtCO2e'),
 (17, 'Metals,Waste', 14.615880466000002, 'MtCO2e'),
 (4, 'Chemic

This looks at the `Minerals` industry (which includes cement).  We see that the top emitters have multiple facility locations.

In [7]:
qres = engine.execute(f"""select count (*), parent_company_name, sum(total_reported_direct_emissions)/1000000, '<- MtCO2e' as metric
from epa_ghgrp.direct_emitters, epa_ghgrp.parent_company
where year>=DATE('2019-01-01') and year<DATE('2020-01-01') and year=reporting_year
      and latest_reported_industry_type_sectors='Minerals'
      and epa_ghgrp.direct_emitters.facility_id=epa_ghgrp.parent_company.ghgrp_facility_id
group by parent_company_name
order by sum(total_reported_direct_emissions) desc
limit 20
""")
display(qres.fetchall())

[(12, 'HOLCIM PARTICIPATIONS (US) INC', 10.775100462, '<- MtCO2e'),
 (9, 'CEMEX INC', 7.837412012, '<- MtCO2e'),
 (12, 'LEHIGH HANSON INC', 7.1708742480000005, '<- MtCO2e'),
 (8, 'RC LONESTAR INC', 6.670842867999999, '<- MtCO2e'),
 (11, 'LHOIST NORTH AMERICA INC', 6.0958935180000005, '<- MtCO2e'),
 (11, 'CARMEUSE LIME INC', 4.957669602, '<- MtCO2e'),
 (9, 'GRAYMONT INC', 4.145991566, '<- MtCO2e'),
 (8, 'CRH AMERICAS INC', 4.038046516, '<- MtCO2e'),
 (4, 'MARTIN MARIETTA MATERIALS INC', 3.9395778740000007, '<- MtCO2e'),
 (4, 'ARGOS USA LLC', 3.571057906, '<- MtCO2e'),
 (11, 'EAGLE MATERIALS INC', 3.46012946, '<- MtCO2e'),
 (3, 'MISSISSIPPI LIME CO', 3.38697296, '<- MtCO2e'),
 (3, 'TAIHEIYO CEMENT USA INC', 3.2686490960000003, '<- MtCO2e'),
 (5, 'GCC OF AMERICA INC', 2.3115211900000006, '<- MtCO2e'),
 (2, 'TITAN AMERICA LLC', 2.23005901, '<- MtCO2e'),
 (1, 'GENESIS ENERGY LP', 1.8242951, '<- MtCO2e'),
 (1, 'LAFARGEHOLCIM NORTH AMERICA INC', 1.7920937160000001, '<- MtCO2e'),
 (2, 'NATIONA

`Suppliers` are those who buy and sell GHG-emitting products, but they do not, themselves, cause the emissions.  They merely enable others to emit.

In [8]:
qres = engine.execute(f"describe epa_ghgrp.suppliers")
display(qres.fetchall())

[('facility_id', 'bigint', '', ''),
 ('frs_id', 'varchar', '', ''),
 ('facility_name', 'varchar', '', ''),
 ('city', 'varchar', '', ''),
 ('state', 'varchar', '', ''),
 ('zip_code', 'varchar', '', ''),
 ('address', 'varchar', '', ''),
 ('county', 'varchar', '', ''),
 ('latitude', 'double', '', ''),
 ('longitude', 'double', '', ''),
 ('primary_naics_code', 'varchar', '', ''),
 ('latest_reported_industry_type_subparts', 'varchar', '', ''),
 ('coal_based_liquid_fuel_production_ghg', 'double', '', ''),
 ('petroleum_products_produced_ghg', 'double', '', ''),
 ('petroleum_products_imported_ghg', 'double', '', ''),
 ('petroleum_products_exported_ghg', 'double', '', ''),
 ('natural_gas_supply_ghg', 'double', '', ''),
 ('natural_gas_liquids_supply_ghg', 'double', '', ''),
 ('co2_supply_ghg', 'double', '', ''),
 ('year', 'timestamp(3)', '', '')]

A quick summary of how many rows of data we have in `epa_ghgrp`.

68k rows in `direct_emitters`: lots of facilities  
103k rows in `parent_company`: lots of facility/owner relationships

In [9]:
qres = engine.execute('show tables in epa_ghgrp')
l = qres.fetchall()
totalrows = 0
for e in l:
    s = f'select count (*) from epa_ghgrp.{e[0]}'
    qres = engine.execute(s)
    rowcount = qres.fetchall()[0][0]
    totalrows += rowcount
    print(f"{rowcount:>6} <- {s})")

print(f'{totalrows} <- total rows')

   954 <- select count (*) from epa_ghgrp.co2_injection)
 68472 <- select count (*) from epa_ghgrp.direct_emitters)
  1703 <- select count (*) from epa_ghgrp.gathering_boosting)
    20 <- select count (*) from epa_ghgrp.geologic_sequestration_of_co2)
  1730 <- select count (*) from epa_ghgrp.ldc_direct_emissions)
  5068 <- select count (*) from epa_ghgrp.onshore_oil_gas_prod)
103043 <- select count (*) from epa_ghgrp.parent_company)
  1012 <- select count (*) from epa_ghgrp.sf6_from_elec_equip)
  8539 <- select count (*) from epa_ghgrp.suppliers)
   780 <- select count (*) from epa_ghgrp.transmission_pipelines)
191321 <- total rows


The key metric is total_emissions (in metric tons of CO2e), but the name of the metric depends on the source/process.  Nevertheless, we know that `year` is our last metric and that the CO2e metric is 2nd-to-last (hence the `-2` index).

We also know that when building our final summary table, the sums feeding into it are all only one row per year.  We use `iat[0,1` to access the 0th row and the 1st column (which will be named specifically to the source/process).  By using `iat`, we get a scalar value we can sum, instead of a Series object we'd have to `squeeze`.

In [10]:
import pandas as pd

emission_tables = ['direct_emitters', 'onshore_oil_gas_prod', 'gathering_boosting', 'transmission_pipelines', 'ldc_direct_emissions', 'sf6_from_elec_equip']
tot_em_columns = []

q_dict = {}

for t in emission_tables:
    qres = engine.execute(f"describe epa_ghgrp.{t}")
    tr = qres.fetchall()
    total_emission_cname = tr[-2][0]
    tot_em_columns.append(total_emission_cname)
    qres = engine.execute(f"select year, sum({total_emission_cname}) from epa_ghgrp.{t} group by year")
    q_dict[t] = pd.DataFrame(qres.fetchall(), columns=['year', total_emission_cname])

# A function that excludes terms using SQL to say "and X!=Y"
def excl_text(excl):
    return ' and '.join([f"latest_reported_industry_type_sectors!='{e}'" for e in excl])

# A function that includes text that matches; SQL that says "or X like '%Y%'"
def incl_text(excl):
    return ' or '.join([f"latest_reported_industry_type_sectors like '%{e}%'" for e in excl])

t = 'direct_emitters'
qres = engine.execute(f"describe epa_ghgrp.{t}")
t_cols = qres.fetchall()
total_emission_cname = t_cols[-2][0]

incl = [ 'Power', 'Petroleum']
qres = engine.execute(f"""
select year, sum({total_emission_cname}) from epa_ghgrp.{t}
where {incl_text(incl)}
group by year
""")
q_dict[t + f" (incl {','.join(incl)})"] = pd.DataFrame(qres.fetchall(), columns=['year', total_emission_cname + f" (matching {','.join(incl)})"])

excl = [ 'Minerals', 'Other', 'Waste', 'Chemicals', 'Pulp and Paper,Waste', 'Metals,Waste', 'Pulp and Paper']
qres = engine.execute(f"""
select year, sum({total_emission_cname}) from epa_ghgrp.{t}
where {excl_text(excl)}
group by year
""")
q_dict[t + f" (excl {','.join(excl)})"] = pd.DataFrame(qres.fetchall(), columns=['year', total_emission_cname + f" (excl {','.join(excl)})"])

for t in emission_tables:
    qres = engine.execute(f"describe epa_ghgrp.{t}")
    tr = qres.fetchall()
    total_emission_cname = tr[-2][0]
    qres = engine.execute(f"select year, sum({total_emission_cname}) from epa_ghgrp.{t} group by year")
    q_dict[t] = pd.DataFrame(qres.fetchall(), columns=['year', total_emission_cname])

grand_total = {}

for year in q_dict['direct_emitters'].year:
    grand_total[year] = sum([q_dict[t][q_dict[t].year==year].iat[0,1] for t in emission_tables if year in q_dict[t].year.values])

df = pd.DataFrame.from_dict(grand_total, orient='index', columns=['total_co2e']).reset_index()
df.rename(columns={'index':'year'}, inplace=True)
q_dict['grand_total'] = df

This gem comes from https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes

In [11]:
from functools import reduce

df_merged = reduce(lambda left,right: pd.merge(left,right,on=['year'], how='outer'), q_dict.values()).fillna(0)

A summary table showing (1) direct emitters that match "Power" or "Petroleum", (2) direct emitters that are not the top other industrie, and then (3..N): the totals from the GHGRP.

In [12]:
df_merged.rename(columns={v:v.replace('_', ' ') for v in df_merged.columns.values}).sort_values(by='year', ascending=False)

,year,total reported direct emissions,total reported emissions from onshore oil gas production,total reported emissions from gathering boosting,total reported direct emissions from transmission pipelines,total reported direct emissions from local dist companies,total reported direct emissions from electrical equipment use,"total reported direct emissions (matching Power,Petroleum)","total reported direct emissions (excl Minerals,Other,Waste,Chemicals,Pulp and Paper,Waste,Metals,Waste,Pulp and Paper)",total co2e
3,2020-01-01 00:00:00.000,2.400335e+09,9.348811e+07,9.002867e+07,3.497590e+06,1.264110e+07,2.004836e+06,1.768353e+09,1.947463e+09,2.601995e+09
2,2019-01-01 00:00:00.000,2.626532e+09,1.201743e+08,9.276566e+07,2.859475e+06,1.284702e+07,2.510832e+06,1.958369e+09,2.153508e+09,2.857689e+09
6,2018-01-01 00:00:00.000,2.779471e+09,1.119588e+08,8.332560e+07,3.050315e+06,1.323626e+07,2.270228e+06,2.099221e+09,2.304343e+09,2.993312e+09
5,2017-01-01 00:00:00.000,2.735841e+09,9.624146e+07,7.783058e+07,2.699047e+06,1.367043e+07,2.555766e+06,2.070082e+09,2.270354e+09,2.928838e+09
0,2016-01-01 00:00:00.000,2.805105e+09,8.689825e+07,8.259701e+07,3.183982e+06,1.400229e+07,2.930497e+06,2.144348e+09,2.337524e+09,2.994717e+09
1,2015-01-01 00:00:00.000,2.939444e+09,1.017485e+08,0.000000e+00,0.000000e+00,1.455831e+07,2.472281e+06,2.261725e+09,2.452045e+09,3.058223e+09
7,2014-01-01 00:00:00.000,3.084069e+09,1.019517e+08,0.000000e+00,0.000000e+00,1.477185e+07,3.220287e+06,2.392070e+09,2.592859e+09,3.204013e+09
9,2013-01-01 00:00:00.000,3.073214e+09,9.795946e+07,0.000000e+00,0.000000e+00,1.516147e+07,3.258298e+06,2.392082e+09,2.593335e+09,3.189593e+09
4,2012-01-01 00:00:00.000,3.058076e+09,9.253966e+07,0.000000e+00,0.000000e+00,1.541235e+07,3.236291e+06,2.376107e+09,2.579361e+09,3.169264e+09
8,2011-01-01 00:00:00.000,3.207583e+09,9.119057e+07,0.000000e+00,0.000000e+00,1.566794e+07,3.920547e+06,2.509918e+09,2.728839e+09,3.318362e+09


Cross-check with ESSD tops-down dataset

A quick look at *just* CO2.  We'll look at CO2e in the next set of cells.

In [13]:
qres = engine.execute("""select year, sector_title, sum(value)/1000000000, '<- GtCO2' as metric from essd.ghg_data
where sector_title='Energy systems' and gas='CO2' and year>DATE('2010-01-01') and year<DATE('2020-01-01') and ISO='USA'
group by year, sector_title, gas order by year desc""")
qres.fetchall()

[('2019-01-01 00:00:00.000', 'Energy systems', 1.9875691487149099, '<- GtCO2'),
 ('2018-01-01 00:00:00.000', 'Energy systems', 2.13019562342906, '<- GtCO2'),
 ('2017-01-01 00:00:00.000', 'Energy systems', 2.1068924743279602, '<- GtCO2'),
 ('2016-01-01 00:00:00.000', 'Energy systems', 2.18062761148211, '<- GtCO2'),
 ('2015-01-01 00:00:00.000', 'Energy systems', 2.27583596104433, '<- GtCO2'),
 ('2014-01-01 00:00:00.000', 'Energy systems', 2.4116766483933207, '<- GtCO2'),
 ('2013-01-01 00:00:00.000', 'Energy systems', 2.4103609320616, '<- GtCO2'),
 ('2012-01-01 00:00:00.000', 'Energy systems', 2.39045090039202, '<- GtCO2'),
 ('2011-01-01 00:00:00.000', 'Energy systems', 2.5147805631782396, '<- GtCO2')]

In [14]:
qres = engine.execute('describe essd.ghg_data')
qres.fetchall()

[('iso', 'varchar', '', ''),
 ('country', 'varchar', '', ''),
 ('region_ar6_6', 'varchar', '', ''),
 ('region_ar6_10', 'varchar', '', ''),
 ('region_ar6_22', 'varchar', '', ''),
 ('region_ar6_dev', 'varchar', '', ''),
 ('sector_title', 'varchar', '', ''),
 ('subsector_title', 'varchar', '', ''),
 ('gas', 'varchar', '', ''),
 ('gwp100_ar5', 'integer', '', ''),
 ('value', 'double', '', ''),
 ('year', 'timestamp(3)', '', '')]

In [15]:
qres = engine.execute('describe essd.gwp100_data')
qres.fetchall()

[('iso', 'varchar', '', ''),
 ('country', 'varchar', '', ''),
 ('region_ar6_6', 'varchar', '', ''),
 ('region_ar6_10', 'varchar', '', ''),
 ('region_ar6_22', 'varchar', '', ''),
 ('region_ar6_dev', 'varchar', '', ''),
 ('sector_title', 'varchar', '', ''),
 ('subsector_title', 'varchar', '', ''),
 ('co2', 'double', '', ''),
 ('ch4', 'double', '', ''),
 ('n2o', 'double', '', ''),
 ('fgas', 'double', '', ''),
 ('ghg', 'double', '', ''),
 ('year', 'timestamp(3)', '', '')]

A look at CO2e (presuming that's what GHG gives us from the GWP100 table) for the category `Energy Systems`.

In [16]:
qres = engine.execute("""select year, sector_title, sum(GHG)/1000000000, '<- GtCO2' as metric from essd.gwp100_data
where sector_title='Energy systems' and year>DATE('2010-01-01') and year<DATE('2020-01-01') and ISO='USA'
group by year, sector_title order by year desc""")
qres.fetchall()

[('2019-01-01 00:00:00.000', 'Energy systems', 2.3492919992192673, '<- GtCO2'),
 ('2018-01-01 00:00:00.000', 'Energy systems', 2.477543798400163, '<- GtCO2'),
 ('2017-01-01 00:00:00.000', 'Energy systems', 2.4515470065038967, '<- GtCO2'),
 ('2016-01-01 00:00:00.000', 'Energy systems', 2.51416792400994, '<- GtCO2'),
 ('2015-01-01 00:00:00.000', 'Energy systems', 2.634213163051593, '<- GtCO2'),
 ('2014-01-01 00:00:00.000', 'Energy systems', 2.7847190114105005, '<- GtCO2'),
 ('2013-01-01 00:00:00.000', 'Energy systems', 2.7790465736036998, '<- GtCO2'),
 ('2012-01-01 00:00:00.000', 'Energy systems', 2.75925382443615, '<- GtCO2'),
 ('2011-01-01 00:00:00.000', 'Energy systems', 2.8909498065313897, '<- GtCO2')]

In [17]:
qres = engine.execute("describe us_census.all_sector_survey_2017")
display(qres.fetchall())
qres = engine.execute("select * from us_census.all_sector_survey_2017 where naics2012='221112'")
display(qres.fetchall())


[('geo_id', 'varchar', '', ''),
 ('name', 'varchar', '', ''),
 ('geo_id_f', 'varchar', '', ''),
 ('naics2012', 'varchar', '', ''),
 ('naics2012_f', 'varchar', '', ''),
 ('naics2012_label', 'varchar', '', ''),
 ('year', 'varchar', '', ''),
 ('estab', 'varchar', '', ''),
 ('rcptot', 'varchar', '', ''),
 ('payann', 'varchar', '', ''),
 ('emp', 'varchar', '', '')]

[('0100000US', 'United States', None, '221112', None, 'Fossil fuel electric power generation', '2012', '1416', '81473633', '7997908', '82071'),
 ('0100000US', 'United States', None, '221112', None, 'Fossil fuel electric power generation', '2017', '1711', '75455040', '8192622', '76058')]

Exercise the connection to NAICS and sector information provided by US Department of Commerce (US_CENSUS)

In [18]:
# qres = engine.execute("select count (*), epa_ghgrp.direct_emitters.year, primary_naics_code from epa_ghgrp.direct_emitters where primary_naics_code='221112' group by epa_ghgrp.direct_emitters.year, primary_naics_code order by count (*) desc limit 20")
# display(qres.fetchall())

qres = engine.execute(f"""
select count (*), epa_ghgrp.direct_emitters.year, primary_naics_code, naics2012_label
from epa_ghgrp.direct_emitters, us_census.all_sector_survey_2017
where primary_naics_code=naics2012 and us_census.all_sector_survey_2017.year='2017' and epa_ghgrp.direct_emitters.year=DATE('2017-01-01')
group by epa_ghgrp.direct_emitters.year, primary_naics_code, naics2012_label
order by count (*) desc limit 20
""")
display(qres.fetchall())

[(1281, '2017-01-01 00:00:00.000', '221112', 'Fossil fuel electric power generation'),
 (1134, '2017-01-01 00:00:00.000', '562212', 'Solid waste landfill'),
 (585, '2017-01-01 00:00:00.000', '486210', 'Pipeline transportation of natural gas'),
 (173, '2017-01-01 00:00:00.000', '325193', 'Ethyl alcohol manufacturing'),
 (141, '2017-01-01 00:00:00.000', '324110', 'Petroleum refineries'),
 (120, '2017-01-01 00:00:00.000', '331110', 'Iron and steel mills and ferroalloy manufacturing'),
 (114, '2017-01-01 00:00:00.000', '322121', 'Paper (except newsprint) mills'),
 (100, '2017-01-01 00:00:00.000', '325199', 'All other basic organic chemical manufacturing'),
 (93, '2017-01-01 00:00:00.000', '327310', 'Cement manufacturing'),
 (79, '2017-01-01 00:00:00.000', '212112', 'Bituminous coal underground mining'),
 (77, '2017-01-01 00:00:00.000', '322130', 'Paperboard mills'),
 (75, '2017-01-01 00:00:00.000', '325211', 'Plastics material and resin manufacturing'),
 (69, '2017-01-01 00:00:00.000', '32

In [19]:
from osc_ingest_trino import *
import pyarrow as pa
import pyarrow.parquet as pq
import json

df = pd.read_sql("""select facility_id, year, latitude, longitude, latest_reported_industry_type_sectors, total_reported_direct_emissions
from epa_ghgrp.direct_emitters""", engine)
df.dropna(how='any', inplace=True)
df.facility_id = df.facility_id.astype('int64')
df.year = df.year.astype('datetime64[ns, UTC]')
df.total_reported_direct_emissions = df.total_reported_direct_emissions.astype('float64')

sector_groupings = [(s[0], len(s)-1, s[1:]) for s in df.latest_reported_industry_type_sectors.str.split(',') if type(s)==list]
from collections import defaultdict
d = defaultdict(list)
for s1, s2, s3 in sector_groupings:
    if s2:
        d[f'{s1} ({s2+1})']  = ','.join([s1] + s3)
    else:
        d[s1] = s1

dinv_map = {v: k for k, v in d.items()}
df['sector_groupings'] = df.latest_reported_industry_type_sectors.map(dinv_map)

In [20]:
for sl in df.latest_reported_industry_type_sectors.str.split(','):
    if type(sl)!=list:
        continue
    # Ensure all primary (and if listed, secondary) sectors are represented
    if f's_{sl[0]}' not in df.columns:
        df[f's_{sl[0]}'] = 0.0
    if len(sl)>1 and f's_{sl[1]}' not in df.columns:
        df[f's_{sl[1]}'] = 0.0

In [21]:
attribution_vector = [ pd.Series([1.0]),
                       pd.Series([2.0/3.0, 1.0/3.0]),
                       pd.Series([0.5, 0.3, 0.2]),
                       pd.Series([0.4, 0.3, 0.2, 0.1]),
                       pd.Series([0.30, 0.25, 0.20, 0.15, 0.10]),
                       pd.Series([0.30, 0.24, 0.19, 0.14, 0.09, 0.04])]

def apply_attribution(x):
    sl = x.latest_reported_industry_type_sectors.split(',')
    # Tertiary sectors not previously mentioned are silently converted to Other, keeping our attribution columns from exploding
    appropriate_columns = set([f's_{s}' if f's_{s}' in x else 's_Other' for s in sl])
    x[ appropriate_columns ] = x.total_reported_direct_emissions * attribution_vector[len(appropriate_columns)-1].values
    return x

df_emitters = df.apply(apply_attribution, axis=1)
enforce_sql_column_names(df_emitters, inplace=True)

In [22]:
df_emitters[df_emitters.latest_reported_industry_type_sectors.str.contains(',')]

,facility_id,year,latitude,longitude,latest_reported_industry_type_sectors,total_reported_direct_emissions,sector_groupings,s_waste,s_power_plants,s_petroleum_and_natural_gas_systems,...,s_industrial_gas_suppliers,s_metals,s_suppliers_of_co2,s_pulp_and_paper,s_petroleum_product_suppliers,s_refineries,s_injection_of_co2,s_natural_gas_and_natural_gas_liquids_suppliers,s_import_and_export_of_equipment_containing_fluorintaed_ghgs,s_coal_based_liquid_fuel_supply
8,1004206,2020-01-01 00:00:00+00:00,34.641667,-87.038611,"Chemicals,Industrial Gas Suppliers",4.401626e+04,Chemicals (2),0.000,0.000000e+00,0.000000,...,29344.176016,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
9,1006665,2020-01-01 00:00:00+00:00,41.755000,-90.284167,"Chemicals,Industrial Gas Suppliers",6.578659e+05,Chemicals (2),0.000,0.000000e+00,0.000000,...,438577.264933,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
10,1004836,2020-01-01 00:00:00+00:00,44.789444,-92.908333,"Chemicals,Industrial Gas Suppliers,Minerals",4.954027e+04,NaN,0.000,0.000000e+00,0.000000,...,24770.136495,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
30,1002627,2020-01-01 00:00:00+00:00,43.499510,-92.917090,"Other,Waste",1.425194e+05,Other (2),47506.450,0.000000e+00,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
33,1004761,2020-01-01 00:00:00+00:00,44.958900,-90.960800,"Other,Suppliers of CO2",6.930084e+04,NaN,0.000,0.000000e+00,0.000000,...,0.000000,0.0,46200.560000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68335,1001822,2011-01-01 00:00:00+00:00,42.047900,-104.189000,"Minerals,Waste",1.017244e+05,Minerals (2),33908.144,0.000000e+00,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
68373,1000912,2011-01-01 00:00:00+00:00,33.016300,-79.928500,"Petroleum and Natural Gas Systems,Power Plants",2.450750e+06,NaN,0.000,1.633833e+06,816916.640667,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
68412,1004143,2011-01-01 00:00:00+00:00,39.551680,-110.815840,"Petroleum and Natural Gas Systems,Suppliers of...",5.946525e+04,Petroleum and Natural Gas Systems (2),0.000,0.000000e+00,19821.750000,...,0.000000,0.0,39643.500000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
68414,1004137,2011-01-01 00:00:00+00:00,32.907723,-97.465856,"Petroleum and Natural Gas Systems,Suppliers of...",5.236050e+04,Petroleum and Natural Gas Systems (2),0.000,0.000000e+00,17453.498667,...,0.000000,0.0,34906.997333,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [23]:
df_emitters[df_emitters.latest_reported_industry_type_sectors.str.count(',')>1]

,facility_id,year,latitude,longitude,latest_reported_industry_type_sectors,total_reported_direct_emissions,sector_groupings,s_waste,s_power_plants,s_petroleum_and_natural_gas_systems,...,s_industrial_gas_suppliers,s_metals,s_suppliers_of_co2,s_pulp_and_paper,s_petroleum_product_suppliers,s_refineries,s_injection_of_co2,s_natural_gas_and_natural_gas_liquids_suppliers,s_import_and_export_of_equipment_containing_fluorintaed_ghgs,s_coal_based_liquid_fuel_supply
10,1004836,2020-01-01 00:00:00+00:00,44.789444,-92.908333,"Chemicals,Industrial Gas Suppliers,Minerals",4.954027e+04,NaN,0.0000,0.0,0.0000,...,24770.136495,0.0,0.0000,0.000,0.0000,0.000000e+00,0.0000,0.0,0.0,0.0
232,1001945,2020-01-01 00:00:00+00:00,44.474074,-95.783525,"Other,Suppliers of CO2,Waste",1.727498e+05,Other (3),34549.9604,0.0,0.0000,...,0.000000,0.0,86374.9010,0.000,0.0000,0.000000e+00,0.0000,0.0,0.0,0.0
253,1002515,2020-01-01 00:00:00+00:00,32.756400,-104.211100,"Injection of CO2,Petroleum and Natural Gas Sys...",9.549250e+04,Injection of CO2 (3),0.0000,0.0,19098.4992,...,0.000000,0.0,47746.2480,0.000,0.0000,0.000000e+00,28647.7488,0.0,0.0,0.0
378,1004861,2020-01-01 00:00:00+00:00,40.556501,-89.666235,"Other,Suppliers of CO2,Waste",1.709771e+05,Other (3),34195.4216,0.0,0.0000,...,0.000000,0.0,85488.5540,0.000,0.0000,0.000000e+00,0.0000,0.0,0.0,0.0
379,1000413,2020-01-01 00:00:00+00:00,40.555000,-89.662800,"Other,Suppliers of CO2,Waste",2.660592e+05,Other (3),53211.8456,0.0,0.0000,...,0.000000,0.0,133029.6140,0.000,0.0000,0.000000e+00,0.0000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68217,1006095,2011-01-01 00:00:00+00:00,44.404000,-89.825500,"Pulp and Paper,Suppliers of CO2,Waste",4.333954e+05,Pulp and Paper (3),86679.0792,0.0,0.0000,...,0.000000,0.0,130018.6188,216697.698,0.0000,0.000000e+00,0.0000,0.0,0.0,0.0
68218,1004874,2011-01-01 00:00:00+00:00,44.398600,-89.826300,"Pulp and Paper,Suppliers of CO2,Waste",1.701224e+04,Pulp and Paper (3),3402.4472,0.0,0.0000,...,0.000000,0.0,5103.6708,8506.118,0.0000,0.000000e+00,0.0000,0.0,0.0,0.0
68232,1007518,2011-01-01 00:00:00+00:00,38.837500,-90.067500,"Chemicals,Petroleum Product Suppliers,Refineri...",3.399126e+06,Chemicals (5),849781.5585,0.0,0.0000,...,0.000000,0.0,509868.9351,0.000,339912.6234,1.019738e+06,0.0000,0.0,0.0,0.0
68239,1002310,2011-01-01 00:00:00+00:00,34.628900,-97.168500,"Chemicals,Petroleum Product Suppliers,Refineries",3.655539e+05,Chemicals (3),0.0000,0.0,0.0000,...,0.000000,0.0,0.0000,0.000,109666.1754,1.827770e+05,0.0000,0.0,0.0,0.0


In [24]:
import uuid
ingest_uuid = str(uuid.uuid4())

custom_meta_key_fields = 'metafields'
custom_meta_key = 'metaset'

from datetime import *
ingest_schema = 'ghgrp_demo'
timestamp = str(datetime.now())

qres = engine.execute(f'create schema if not exists {ingest_schema}')
qres.fetchall()

[(True,)]

In [25]:
def create_trino_pipeline (s3, schemaname, tablename, timestamp, df, meta_fields, meta_content):
    global ingest_uuid
    global custom_meta_key_fields, custom_meta_key
    
    # First convert dataframe to pyarrow for type conversion and basic metadata
    table = pa.Table.from_pandas(df)
    # Second, since pyarrow tables are immutable, create a new table with additional combined metadata
    if meta_fields or meta_content:
        meta_json_fields = json.dumps(meta_fields)
        meta_json = json.dumps(meta_content)
        existing_meta = table.schema.metadata
        combined_meta = {
            custom_meta_key_fields.encode(): meta_json_fields.encode(),
            custom_meta_key.encode(): meta_json.encode(),
            **existing_meta
        }
        table = table.replace_schema_metadata(combined_meta)
    # Third, convert table to parquet format (which cannot be written directly to s3)
    pq.write_table(table, f'/tmp/{schemaname}.{tablename}.{ingest_uuid}.{timestamp}.parquet')
    # df.to_parquet(f'/tmp/{schemaname}.{tablename}.{uuid}.parquet', index=False)
    # Fourth, put the parquet-ified data into our S3 bucket for trino.  We cannot compute parquet format directly to S3 but we can copy it once computed
    s3.upload_file(
        Bucket=os.environ['S3_DEV_BUCKET'],
        Key=f'trino/{schemaname}/{tablename}/{ingest_uuid}/{timestamp}/data.parquet',
        Filename=f'/tmp/{schemaname}.{tablename}.{ingest_uuid}.{timestamp}.parquet'
    )
    # Finally, create the trino table backed by our parquet files enhanced by our metadata
    qres = engine.execute(f'drop table if exists {schemaname}.{tablename}')
    print(f'dropping table: {tablename}')
    qres.fetchall()
    
    columnschema = create_table_schema_pairs(df)

    tabledef = f"""create table if not exists {schemaname}.{tablename}(
{columnschema}
) with (
    format = 'parquet',
    external_location = 's3a://{os.environ['S3_DEV_BUCKET']}/trino/{schemaname}/{tablename}/{ingest_uuid}/{timestamp}'
)"""
    print(tabledef)

    # tables created externally may not show up immediately in cloud-beaver
    qres = engine.execute(tabledef)
    qres.fetchall()

In [26]:
import boto3

# Create an S3 client.  We will user later when we write out data and metadata
s3_trino = boto3.client(
    service_name="s3",
    endpoint_url=os.environ['S3_DEV_ENDPOINT'],
    aws_access_key_id=os.environ['S3_DEV_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_DEV_SECRET_KEY'],
)

In [27]:
custom_meta_content = { 'foo':'bar'}
custom_meta_fields = { 'GHGRP FACILITY ID':'GHGRP FACILITY ID'}

tablename = 'direct_emitters_attributed'
create_trino_pipeline (s3_trino, ingest_schema, tablename, timestamp, df_emitters, custom_meta_fields, custom_meta_content)

dropping table: direct_emitters_attributed
create table if not exists ghgrp_demo.direct_emitters_attributed(
    facility_id bigint,
    year timestamp,
    latitude double,
    longitude double,
    latest_reported_industry_type_sectors varchar,
    total_reported_direct_emissions double,
    sector_groupings varchar,
    s_waste double,
    s_power_plants double,
    s_petroleum_and_natural_gas_systems double,
    s_minerals double,
    s_other double,
    s_chemicals double,
    s_industrial_gas_suppliers double,
    s_metals double,
    s_suppliers_of_co2 double,
    s_pulp_and_paper double,
    s_petroleum_product_suppliers double,
    s_refineries double,
    s_injection_of_co2 double,
    s_natural_gas_and_natural_gas_liquids_suppliers double,
    s_import_and_export_of_equipment_containing_fluorintaed_ghgs double,
    s_coal_based_liquid_fuel_supply double
) with (
    format = 'parquet',
    external_location = 's3a://ocp-odh-os-demo-s3/trino/ghgrp_demo/direct_emitters_attribu

Here's an example of a facility with many owners...

In [28]:
qres = engine.execute("""select ghgrp_facility_id,frs_id_facility,lei,reporting_year,facility_name,facility_city,facility_state,parent_company_name,facility_naics_code
from epa_ghgrp.parent_company where reporting_year=DATE('2020-01-01') and ghgrp_facility_id=1005071 order by lei""")
qres.fetchall()

[(1005071, '110000702730', '2549000NXAL5JJHJYT18', '2020-01-01 00:00:00.000', 'North Terrebonne Gas Plant', 'Gibson', 'LA', 'ENERGY RESOURCES TECHNOLOGY LAND INC', '211130'),
 (1005071, '110000702730', '54930000S35EESPK1C27', '2020-01-01 00:00:00.000', 'North Terrebonne Gas Plant', 'Gibson', 'LA', 'BYRON ENERGY LLC', '211130'),
 (1005071, '110000702730', '5493003QENHHS261UR94', '2020-01-01 00:00:00.000', 'North Terrebonne Gas Plant', 'Gibson', 'LA', 'TARGA RESOURCES CORP', '211130'),
 (1005071, '110000702730', '5493005Y7TJPYWLDEO18', '2020-01-01 00:00:00.000', 'North Terrebonne Gas Plant', 'Gibson', 'LA', 'ARENA ENERGY LP', '211130'),
 (1005071, '110000702730', '5493007VQUSLFRDRBT52', '2020-01-01 00:00:00.000', 'North Terrebonne Gas Plant', 'Gibson', 'LA', 'SUPERIOR NATURAL GAS CORP', '211130'),
 (1005071, '110000702730', '549300HX0ISXOOEMR657', '2020-01-01 00:00:00.000', 'North Terrebonne Gas Plant', 'Gibson', 'LA', 'BLACK ELK ENERGY OFFSHORE OPERATIONS LLC', '211130'),
 (1005071, '11

...meaning 10 rows of data that's outside our easy-to-aggregate data

In [29]:
qres = engine.execute("""
select facility_id,facility_name,total_reported_direct_emissions,city,state,latitude,longitude,primary_naics_code,
latest_reported_industry_type_subparts,latest_reported_industry_type_sectors,year
from epa_ghgrp.direct_emitters where facility_id=1005071 order by year""")
qres.fetchall()

[(1005071, 'North Terrebonne Gas Plant', 383446.646, 'Gibson', 'LA', 29.6257, -90.9289, '211130', 'C,W-PROC', 'Petroleum and Natural Gas Systems', '2011-01-01 00:00:00.000'),
 (1005071, 'North Terrebonne Gas Plant', 339163.524, 'Gibson', 'LA', 29.6257, -90.9289, '211130', 'C,W-PROC', 'Petroleum and Natural Gas Systems', '2012-01-01 00:00:00.000'),
 (1005071, 'North Terrebonne Gas Plant', 313640.418, 'Gibson', 'LA', 29.6257, -90.9289, '211130', 'C,W-PROC', 'Petroleum and Natural Gas Systems', '2013-01-01 00:00:00.000'),
 (1005071, 'North Terrebonne Gas Plant', 312585.924, 'Gibson', 'LA', 29.6257, -90.9289, '211130', 'C,W-PROC', 'Petroleum and Natural Gas Systems', '2014-01-01 00:00:00.000'),
 (1005071, 'North Terrebonne Gas Plant', 292713.398, 'Gibson', 'LA', 29.6257, -90.9289, '211130', 'C,W-PROC', 'Petroleum and Natural Gas Systems', '2015-01-01 00:00:00.000'),
 (1005071, 'North Terrebonne Gas Plant', 216519.086, 'Gibson', 'LA', 29.6257, -90.9289, '211130', 'C,W-PROC', 'Petroleum and 

In [30]:
qres = engine.execute('drop table if exists ghgrp_demo.parent_attribution')
qres.fetchall()

display([(x, y) for x, y in zip(emission_tables,tot_em_columns)])

emission_selects = [ f"""
select ghgrp_facility_id, reporting_year, lei, '{e_tbl}' as table_source,
         parent_co_percent_ownership * 0.01 * {e_col} as fractional_emissions
    from epa_ghgrp.parent_company, epa_ghgrp.{e_tbl}
    where epa_ghgrp.parent_company.ghgrp_facility_id=epa_ghgrp.{e_tbl}.facility_id
          and epa_ghgrp.parent_company.reporting_year=epa_ghgrp.{e_tbl}.year
""" for e_tbl, e_col in zip(emission_tables,tot_em_columns) ]


sql = f"""
create table ghgrp_demo.parent_attribution as {' union all '.join(emission_selects)}
"""

print(sql)

qres = engine.execute(sql)
qres.fetchall()

[('direct_emitters', 'total_reported_direct_emissions'),
 ('onshore_oil_gas_prod',
  'total_reported_emissions_from_onshore_oil_gas_production'),
 ('gathering_boosting', 'total_reported_emissions_from_gathering_boosting'),
 ('transmission_pipelines',
  'total_reported_direct_emissions_from_transmission_pipelines'),
 ('ldc_direct_emissions',
  'total_reported_direct_emissions_from_local_dist_companies'),
 ('sf6_from_elec_equip',
  'total_reported_direct_emissions_from_electrical_equipment_use')]


create table ghgrp_demo.parent_attribution as 
select ghgrp_facility_id, reporting_year, lei, 'direct_emitters' as table_source,
         parent_co_percent_ownership * 0.01 * total_reported_direct_emissions as fractional_emissions
    from epa_ghgrp.parent_company, epa_ghgrp.direct_emitters
    where epa_ghgrp.parent_company.ghgrp_facility_id=epa_ghgrp.direct_emitters.facility_id
          and epa_ghgrp.parent_company.reporting_year=epa_ghgrp.direct_emitters.year
 union all 
select ghgrp_facility_id, reporting_year, lei, 'onshore_oil_gas_prod' as table_source,
         parent_co_percent_ownership * 0.01 * total_reported_emissions_from_onshore_oil_gas_production as fractional_emissions
    from epa_ghgrp.parent_company, epa_ghgrp.onshore_oil_gas_prod
    where epa_ghgrp.parent_company.ghgrp_facility_id=epa_ghgrp.onshore_oil_gas_prod.facility_id
          and epa_ghgrp.parent_company.reporting_year=epa_ghgrp.onshore_oil_gas_prod.year
 union all 
select ghgrp_facility_id, reporting_year,

[(89879,)]

In [31]:
qres = engine.execute('describe ghgrp_demo.parent_attribution')
display(qres.fetchall())

qres = engine.execute("select *, '<- t CO2e' as metric from ghgrp_demo.parent_attribution")
qres.fetchall()[::2000]

[('ghgrp_facility_id', 'bigint', '', ''),
 ('reporting_year', 'timestamp(3)', '', ''),
 ('lei', 'varchar', '', ''),
 ('table_source', 'varchar(22)', '', ''),
 ('fractional_emissions', 'double', '', '')]

[(1004919, '2014-01-01 00:00:00.000', '549300ZDYXT3JUQQ9C89', 'ldc_direct_emissions', 99725.0, '<- t CO2e'),
 (1012378, '2019-01-01 00:00:00.000', '549300EV3SSPE7QET085', 'gathering_boosting', 867757.466, '<- t CO2e'),
 (1012352, '2020-01-01 00:00:00.000', '549300GGJCRSI2TIEJ46', 'direct_emitters', 60628.684, '<- t CO2e'),
 (1001380, '2020-01-01 00:00:00.000', '549300R8DACS3G0K8262', 'direct_emitters', 182808.572, '<- t CO2e'),
 (1013637, '2020-01-01 00:00:00.000', None, 'direct_emitters', 12302.9, '<- t CO2e'),
 (1006829, '2014-01-01 00:00:00.000', '549300HKQQ9XYFEUZM13', 'direct_emitters', 50042.612, '<- t CO2e'),
 (1003578, '2013-01-01 00:00:00.000', 'XAP7J9TZOQWCQJWQ7163', 'direct_emitters', 72133.564, '<- t CO2e'),
 (1001319, '2012-01-01 00:00:00.000', 'VR27ZYPWHGW7Z1BM8Y69', 'direct_emitters', 24723.28931862116, '<- t CO2e'),
 (1008042, '2012-01-01 00:00:00.000', '5493005158BXZWGOX572', 'direct_emitters', 281803.144, '<- t CO2e'),
 (1003152, '2017-01-01 00:00:00.000', '5493008K8W

How many **_facilities owned by public companies_** match to corporate reports we can see using the SEC's DERA dataset?

See how many `PARENT_COMPANY` records have LEIs we know.  Note that there are about 8400 total facilities, so 4 facilities not covered by LEI for each that is.
There are 3K-4K distinctly named entities, so average entity owns (at least partially) approx 2-3 facilities.  It also means we know the LEIs of approximately half of the parent copmanies.

In [32]:
qres = engine.execute(f"""select count (*), reporting_year
from (select lei, reporting_year from epa_ghgrp.parent_company where LEI is not null group by lei, reporting_year)
group by reporting_year order by reporting_year desc""")
qres.fetchall()

[(1542, '2020-01-01 00:00:00.000'),
 (1559, '2019-01-01 00:00:00.000'),
 (1626, '2018-01-01 00:00:00.000'),
 (1562, '2017-01-01 00:00:00.000'),
 (1560, '2016-01-01 00:00:00.000'),
 (1541, '2015-01-01 00:00:00.000'),
 (1712, '2014-01-01 00:00:00.000'),
 (1695, '2013-01-01 00:00:00.000'),
 (1657, '2012-01-01 00:00:00.000'),
 (1578, '2011-01-01 00:00:00.000'),
 (1309, '2010-01-01 00:00:00.000')]

In [33]:
qres = engine.execute(f"describe sec_dera.sub")
qres.fetchall()

[('adsh', 'varchar', '', ''),
 ('cik', 'integer', '', ''),
 ('name', 'varchar', '', ''),
 ('lei', 'varchar', '', ''),
 ('sic', 'integer', '', ''),
 ('countryba', 'varchar', '', ''),
 ('stprba', 'varchar', '', ''),
 ('cityba', 'varchar', '', ''),
 ('zipba', 'varchar', '', ''),
 ('bas1', 'varchar', '', ''),
 ('bas2', 'varchar', '', ''),
 ('baph', 'varchar', '', ''),
 ('countryma', 'varchar', '', ''),
 ('stprma', 'varchar', '', ''),
 ('cityma', 'varchar', '', ''),
 ('zipma', 'varchar', '', ''),
 ('mas1', 'varchar', '', ''),
 ('mas2', 'varchar', '', ''),
 ('countryinc', 'varchar', '', ''),
 ('stprinc', 'varchar', '', ''),
 ('ein', 'bigint', '', ''),
 ('former', 'varchar', '', ''),
 ('changed', 'varchar', '', ''),
 ('afs', 'varchar', '', ''),
 ('wksi', 'boolean', '', ''),
 ('fye', 'varchar', '', ''),
 ('form', 'varchar', '', ''),
 ('period', 'timestamp(3)', '', ''),
 ('fy', 'timestamp(3)', '', ''),
 ('fp', 'varchar', '', ''),
 ('filed', 'timestamp(3)', '', ''),
 ('accepted', 'timestamp(3)',

In [34]:
qres = engine.execute(f"describe sec_dera.num")
qres.fetchall()

[('adsh', 'varchar', '', ''),
 ('tag', 'varchar', '', ''),
 ('version', 'varchar', '', ''),
 ('coreg', 'varchar', '', ''),
 ('ddate', 'timestamp(3)', '', ''),
 ('qtrs', 'integer', '', ''),
 ('uom', 'varchar', '', ''),
 ('value', 'double', '', ''),
 ('footnote', 'varchar', '', ''),
 ('uuid', 'varchar', '', ''),
 ('quarter', 'varchar', '', '')]

In [35]:
qres = engine.execute(f"""select count (*), ghgrp_demo.parent_attribution.reporting_year
from ghgrp_demo.parent_attribution, sec_dera.sub
where form='10-K'
and reporting_year >= DATE('2020-01-01') and filed >= DATE('2020-01-01') and filed < DATE('2021-01-01')
and ghgrp_demo.parent_attribution.lei=sec_dera.sub.lei and ghgrp_demo.parent_attribution.lei is not null
group by ghgrp_demo.parent_attribution.reporting_year
order by ghgrp_demo.parent_attribution.reporting_year
""")
qres.fetchall()

[(3711, '2020-01-01 00:00:00.000')]

We can tie these companies to ticker symbols...

In [36]:
qres = engine.execute(f"""select * from sec_dera.ticker limit 10""")
qres.fetchall()

[(320193, 'aapl'),
 (789019, 'msft'),
 (1652044, 'goog'),
 (1018724, 'amzn'),
 (1318605, 'tsla'),
 (1326801, 'fb'),
 (40545, 'ge'),
 (1067983, 'brk-a'),
 (1046179, 'tsm'),
 (1293451, 'tcehy')]

How many distinct companies own these facilities (and what are their ticker symbols)?

In [37]:
qres = engine.execute(f"""
with leis as (select DISTINCT(sec_dera.sub.lei), name, if(tname IS NULL, '<private>', tname) as ticker
              from ghgrp_demo.parent_attribution,sec_dera.sub,sec_dera.ticker
              where ghgrp_demo.parent_attribution.lei=sec_dera.sub.lei and form='10-K'
              and sec_dera.sub.cik=sec_dera.ticker.cik
              and period>=DATE('2020-01-01')
              and period<DATE('2021-01-01'))
select count (*), ticker, leis.lei, name, reporting_year
from ghgrp_demo.parent_attribution, leis
where reporting_year >= DATE('2020-01-01')
and ghgrp_demo.parent_attribution.lei=leis.lei
group by leis.ticker, leis.lei, name, reporting_year
order by count(*) desc
-- limit 10
""")
ticker_list = qres.fetchall()
print(len(ticker_list))

417


Note that some comapnies have more than one ticker symbol!

In [38]:
ticker_list[0:50]

[(259, 'kmi', '549300WR7IX8XE0TBO16', 'KINDER MORGAN, INC.', '2020-01-01 00:00:00.000'),
 (259, 'ep-pc', '549300WR7IX8XE0TBO16', 'KINDER MORGAN, INC.', '2020-01-01 00:00:00.000'),
 (228, 'wm', '549300YX8JIID70NFS41', 'WASTE MANAGEMENT INC', '2020-01-01 00:00:00.000'),
 (162, 'rsg', 'NKNQHM6BLECKVOQP7O46', 'REPUBLIC SERVICES, INC.', '2020-01-01 00:00:00.000'),
 (158, 'et-pe', 'MTLVN9N7JE8MIBIJ1H73', 'ENERGY TRANSFER LP', '2020-01-01 00:00:00.000'),
 (158, 'et-pd', 'MTLVN9N7JE8MIBIJ1H73', 'ENERGY TRANSFER LP', '2020-01-01 00:00:00.000'),
 (158, 'et-pc', 'MTLVN9N7JE8MIBIJ1H73', 'ENERGY TRANSFER LP', '2020-01-01 00:00:00.000'),
 (158, 'et', 'MTLVN9N7JE8MIBIJ1H73', 'ENERGY TRANSFER LP', '2020-01-01 00:00:00.000'),
 (114, 'brk-a', '5493000C01ZX7D35SD85', 'BERKSHIRE HATHAWAY INC', '2020-01-01 00:00:00.000'),
 (114, 'brk-b', '5493000C01ZX7D35SD85', 'BERKSHIRE HATHAWAY INC', '2020-01-01 00:00:00.000'),
 (89, 'xom', 'J3WHBG0MTS7O8ZVMDC91', 'EXXON MOBIL CORP', '2020-01-01 00:00:00.000'),
 (87, 'w

We can try to add up all the faciltiies for all the tickers, but that leads to counting duplicates for companies that have multiple ticker symbols...(should be 2651, not 3935)

In [39]:
sum([te[0] for te in ticker_list])

5746

Sample data to cross-check LEI, Facility ID and EDGAR submission data

In [40]:
qres = engine.execute(f"""
select DISTINCT(sec_dera.sub.lei), ghgrp_facility_id, adsh
              from ghgrp_demo.parent_attribution,sec_dera.sub
              where reporting_year>=DATE('2020-01-01')
              and ghgrp_demo.parent_attribution.lei=sec_dera.sub.lei and form='10-K'
              and period>=DATE('2020-01-01') and period<DATE('2021-01-01')""")
l = qres.fetchall()
print(len(l))
display(l[::60])

3635


[('549300OQS2LO07ZJ7N73', 1004259, '0000022444-20-000047'),
 ('5493000C01ZX7D35SD85', 1000223, '0001564590-21-009611'),
 ('I1BZKREC126H0VB1BL91', 1005857, '0001326160-21-000063'),
 ('NKNQHM6BLECKVOQP7O46', 1004483, '0001060391-21-000014'),
 ('IM7X0T3ECJW4C1T7ON55', 1011917, '0000797468-21-000009'),
 ('WD6L6041MNRW1JE49D58', 1007915, '0000100493-20-000132'),
 ('549300C841GHSUPKR373', 1000591, '0001637757-21-000004'),
 ('5493005JBO5YSIGK1814', 1007506, '0001534701-21-000074'),
 ('52990028CHK9KUB1B293', 1009197, '0001255474-21-000009'),
 ('5493003X5Z6TVAXIPW56', 1011462, '0001045309-21-000028'),
 ('K05C0SER542GQ6VLRO68', 1006435, '0001564590-20-055198'),
 ('5493003QENHHS261UR94', 1005071, '0001564590-21-006640'),
 ('549300WR7IX8XE0TBO16', 1012474, '0001506307-21-000022'),
 ('3SOUA6IRML7435B56G12', 1003769, '0001109357-21-000022'),
 ('TWSEY0NEDUDCKS27AH81', 1007299, '0000764622-21-000013'),
 ('549300FC3G3YU2FBZD92', 1009889, '0000092122-21-000006'),
 ('549300C841GHSUPKR373', 1004854, '0001

Compute intensity in metric tons of CO2e per million dollars

In [41]:
qres = engine.execute(f"""
with leis as (select DISTINCT(sec_dera.sub.lei), ghgrp_facility_id, name, adsh
              from ghgrp_demo.parent_attribution,sec_dera.sub
              where reporting_year>=DATE('2020-01-01')
              and ghgrp_demo.parent_attribution.lei=sec_dera.sub.lei and form='10-K'
              and period>=DATE('2020-01-01') and period<DATE('2021-01-01'))
select leis.lei as lei, name, sum(fractional_emissions) as tot_co2e, reporting_year, uom,
       sum(value) as tot_revenue, 1000000*sum(fractional_emissions)/sum(value) as intensity,
       '<- tCO2e / $MM' as intensity_metric
from leis, ghgrp_demo.parent_attribution, sec_dera.num
where ghgrp_demo.parent_attribution.ghgrp_facility_id=leis.ghgrp_facility_id
and ghgrp_demo.parent_attribution.lei=leis.lei
and reporting_year >= DATE('2020-01-01')
and ddate>=DATE('2020-01-01') and ddate<DATE('2021-01-01')
and leis.adsh=sec_dera.num.adsh
and sec_dera.num.tag = 'Revenues'
and sec_dera.num.qtrs=4
and fractional_emissions>0
group by leis.lei, name, reporting_year, uom
order by intensity desc
-- limit 100
""")
list = qres.fetchall()
print(len(list))
display(list[::5])

180


[('549300O4B5CVWMKUES27', 'MIDWEST ENERGY EMISSIONS CORP.', 29981.588, '2020-01-01 00:00:00.000', 'USD', 8158448.0, 3674.9131697597386, '<- tCO2e / $MM'),
 ('549300F0PA3NW6D2OO63', 'HIGHPOINT RESOURCES CORP', 286194.984, '2020-01-01 00:00:00.000', 'USD', 250347000.0, 1143.1931838608011, '<- tCO2e / $MM'),
 ('9N3UAJSNOUXFKQLF3V18', 'PPL CORP', 134850910.72000003, '2020-01-01 00:00:00.000', 'USD', 145710000000.0, 925.4746463523439, '<- tCO2e / $MM'),
 ('549300YON1UVV8QXPK98', 'CARDINAL ETHANOL LLC', 176449.336, '2020-01-01 00:00:00.000', 'USD', 244718562.0, 721.0296373022983, '<- tCO2e / $MM'),
 ('549300PGTHDQY6PSUI61', 'EVERGY, INC.', 71825943.45368122, '2020-01-01 00:00:00.000', 'USD', 126519400000.0, 567.7069560374237, '<- tCO2e / $MM'),
 ('549300CI3Q2UUK5R4G79', 'MATADOR RESOURCES CO', 800681.748, '2020-01-01 00:00:00.000', 'USD', 1724252000.0, 464.3646914720122, '<- tCO2e / $MM'),
 ('5493003S4AVU6OSXB118', 'PENN VIRGINIA CORP', 110275.084, '2020-01-01 00:00:00.000', 'USD', 273268000

In [42]:
qreg=engine.execute("""select DISTINCT(sec_dera.sub.lei), ghgrp_facility_id, name, adsh
              from ghgrp_demo.parent_attribution,sec_dera.sub
              where reporting_year>=DATE('2020-01-01') and sec_dera.sub.lei='549300O4B5CVWMKUES27'
              and ghgrp_demo.parent_attribution.lei=sec_dera.sub.lei and form='10-K'
              and period>=DATE('2020-01-01') and period<DATE('2021-01-01')""")
qreg.fetchall()

[('549300O4B5CVWMKUES27', 1012016, 'MIDWEST ENERGY EMISSIONS CORP.', '0001477932-21-002039')]

In [43]:
qreg=engine.execute("""select reporting_year, sum(fractional_emissions), '<- t CO2e' as metric
              from ghgrp_demo.parent_attribution
              where lei='549300O4B5CVWMKUES27'
              group by reporting_year
""")
l = qreg.fetchall()
l

[('2016-01-01 00:00:00.000', 98413.61, '<- t CO2e'),
 ('2015-01-01 00:00:00.000', 46877.452000000005, '<- t CO2e'),
 ('2019-01-01 00:00:00.000', 69246.51000000001, '<- t CO2e'),
 ('2018-01-01 00:00:00.000', 100039.14799999999, '<- t CO2e'),
 ('2020-01-01 00:00:00.000', 29981.588, '<- t CO2e'),
 ('2012-01-01 00:00:00.000', 24854.15, '<- t CO2e'),
 ('2017-01-01 00:00:00.000', 71698.954, '<- t CO2e'),
 ('2011-01-01 00:00:00.000', 25032.1, '<- t CO2e'),
 ('2014-01-01 00:00:00.000', 21395.3, '<- t CO2e'),
 ('2013-01-01 00:00:00.000', 24847.4, '<- t CO2e')]

GHGRP Direct Emitters include Cement and Steel Plans (which we can connect to SFI data)

In [44]:
qres = engine.execute("describe sfi_geoasset.cement")
display(qres.fetchall())

[('uid', 'varchar', '', ''),
 ('city', 'varchar', '', ''),
 ('state', 'varchar', '', ''),
 ('country', 'varchar', '', ''),
 ('iso3', 'varchar', '', ''),
 ('country_code', 'double', '', ''),
 ('region', 'varchar', '', ''),
 ('sub_region', 'varchar', '', ''),
 ('latitude', 'double', '', ''),
 ('longitude', 'double', '', ''),
 ('accuracy', 'varchar', '', ''),
 ('status', 'varchar', '', ''),
 ('plant_type', 'varchar', '', ''),
 ('production_type', 'varchar', '', ''),
 ('capacity', 'double', '', ''),
 ('capacity_source', 'varchar', '', ''),
 ('year', 'timestamp(3)', '', ''),
 ('owner_permid', 'double', '', ''),
 ('owner_name', 'varchar', '', ''),
 ('owner_source', 'varchar', '', ''),
 ('parent_permid', 'double', '', ''),
 ('parent_name', 'varchar', '', ''),
 ('ownership_stake', 'double', '', ''),
 ('parent_lei', 'varchar', '', ''),
 ('parent_holding_status', 'varchar', '', ''),
 ('parent_ticker', 'varchar', '', ''),
 ('parent_exchange', 'varchar', '', ''),
 ('parent_permid_2', 'double', '',

In [45]:
qres = engine.execute("select count (*) from sfi_geoasset.cement")
display(qres.fetchall())
qres = engine.execute("select count (*) from sfi_geoasset.steel")
display(qres.fetchall())

# There are 105 US-located cement plants listed in the SFI report with parent LEIs
qres = engine.execute("select count (*), iso3 from sfi_geoasset.cement where iso3='USA' group by iso3")
display(qres.fetchall())

qres = engine.execute("""
select owner_name, parent_name, lei, parent_lei, facility_id
from sfi_geoasset.cement, epa_ghgrp.direct_emitters, epa_ghgrp.parent_company
where ghgrp_facility_id=facility_id
and reporting_year=epa_ghgrp.direct_emitters.year
and reporting_year>=DATE('2019-01-01') and reporting_year<DATE('2020-01-01')
and sfi_geoasset.cement.iso3='USA'
and abs(sfi_geoasset.cement.latitude-epa_ghgrp.direct_emitters.latitude)<0.01
and abs(sfi_geoasset.cement.longitude-epa_ghgrp.direct_emitters.longitude)<0.01
""")
l = qres.fetchall()
print(f"{len(l)}: facilities/parent relationships matched in USA using lat/lon")

[(3117,)]

[(1598,)]

[(105, 'USA')]

108: facilities/parent relationships matched in USA using lat/lon


In [46]:
l[3::2]

[('Ash Grove Cement Co', 'CRH PLC', '549300RN11MJ182CNF63', '549300MIDJNNTH068E74', 1002530),
 ('Capitol Aggregates Inc', 'Capitol Aggregates Inc', '549300ROAT6R8JSEVV96', '549300W2BSPN5YV5JS04', 1004270),
 ('Cemex Inc', 'CEMEX SAB de CV', '549300JHGUF0VVA38719', '549300RIG2CXWN6IV731', 1004758),
 ('CalPortland Co', 'Taiheiyo Cement Corp', None, None, 1006669),
 ('Fairborn Cement Company LLC', 'Eagle Cement Corp', '529900KRXRGXL61SG039', None, 1006212),
 ('Grupo Cementos de Chihuahua SAB de CV', 'Grupo Cementos de Chihuahua SAB de CV', None, '254900WY9XL8033CKX68', 1009018),
 ('Lafarge North America Inc', 'LafargeHolcim Ltd', '5299007BR966QQRQTB48', '529900EHPFPYHV6IQO98', 1006815),
 ('Lafarge North America Inc', 'LafargeHolcim Ltd', None, '529900EHPFPYHV6IQO98', 1005874),
 ('Lehigh White Cement Co', 'Cementir Holding NV', None, '8156008B101B97A43B02', 1002049),
 ('Lehigh Hanson Inc', 'HeidelbergCement AG', 'LZ2C6E0W5W7LQMX5ZI37', 'LZ2C6E0W5W7LQMX5ZI37', 1006136),
 ('Lehigh Hanson Inc'